In [84]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from datasets import load_dataset
import pandas as pd
import ast
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re





In [38]:
#dowlnoad the dataset
data_load = load_dataset("nedjmaou/MLMA_hate_speech")
data_train = data_load["train"]
dataset = data_train.to_pandas()
print(dataset.head())
print(dataset.shape)

   HITId                                              tweet       sentiment  \
0      1  صلاة الفجر خير لك من ترديد بول البعير وسبي الن...  hateful_normal   
1      2  صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...       offensive   
2      3  طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...       offensive   
3      4  @user @user انا اوافقك بخصوص السوريين و العراق...          normal   
4      5  هذه السعودية التي شعبها شعب الخيم و بول البعير...          normal   

  directness                           annotator_sentiment  target       group  
0   indirect                                         shock  gender  individual  
1   indirect  anger_confusion_sadness_indifference_disgust   other       other  
2   indirect                                  indifference   other  individual  
3     direct                                  indifference  origin       other  
4   indirect                                  indifference  origin       other  
(18661, 7)


In [69]:
#dataset separate by languages
arab_path = "/Users/benoitmathey-doret/Documents/EPFL/MA2/DL/project/deeplearning/dataset/ar_dataset_600.csv"
ar_dataset = pd.read_csv(arab_path)
print(ar_dataset.shape)
print(ar_dataset.head())
ar_dataset = ar_dataset.sample(frac=1, random_state=42)



en_path="/Users/benoitmathey-doret/Documents/EPFL/MA2/DL/project/deeplearning/dataset/en_dataset_600.csv"
en_dataset = pd.read_csv(en_path)
print(en_dataset.shape)
print(en_dataset.head())
en_dataset = en_dataset.sample(frac=1, random_state=42)


fr_path="/Users/benoitmathey-doret/Documents/EPFL/MA2/DL/project/deeplearning/dataset/fr_dataset_600.csv"
fr_dataset = pd.read_csv(fr_path)
print(fr_dataset.shape)
print(fr_dataset.head())
fr_dataset = fr_dataset.sample(frac=1, random_state=42)

#create binary datasets
en_dataset_binary=en_dataset.copy()
en_dataset_binary = en_dataset_binary[en_dataset_binary['sentiment_binary'].isin([2, 3])]
en_dataset_binary['sentiment_binary'] = en_dataset_binary['sentiment_binary'].replace(2, 0)
en_dataset_binary['sentiment_binary'] = en_dataset_binary['sentiment_binary'].replace(3, 1)

ar_dataset_binary=ar_dataset.copy()
ar_dataset_binary = ar_dataset_binary[ar_dataset_binary['sentiment_binary'].isin([2, 3])]
ar_dataset_binary['sentiment_binary'] = ar_dataset_binary['sentiment_binary'].replace(2, 0)
ar_dataset_binary['sentiment_binary'] = ar_dataset_binary['sentiment_binary'].replace(3, 1)

fr_dataset_binary=fr_dataset.copy()
fr_dataset_binary = fr_dataset_binary[fr_dataset_binary['sentiment_binary'].isin([2, 3])]
fr_dataset_binary['sentiment_binary'] = fr_dataset_binary['sentiment_binary'].replace(2, 0)
fr_dataset_binary['sentiment_binary'] = fr_dataset_binary['sentiment_binary'].replace(3, 1)




(600, 8)
   HITId                                              tweet sentiment  \
0   1486                          بول البعير وعلاج السرطان!    normal   
1   3194  الإسلام يتعامل مع القضايا قبل حصولها فيسعى لعل...    normal   
2   1384  @user @user معلش سؤال..الكائن الابيض دا نوعه ا...    normal   
3   1391  محاضرة التحرش والابتزاز كلية الحقوق والعلوم ال...    normal   
4   1420  حرب شرسه على المرآه وحجابها من كل مكان الاعلام...    normal   

  directness   annotator_sentiment    target  group  sentiment_binary  
0   indirect  indifference_disgust     other  other                 1  
1   indirect          indifference  religion  other                 1  
2   indirect                 shock    origin  other                 1  
3   indirect          indifference     other  other                 1  
4     direct               sadness     other  women                 1  
(600, 8)
   HITId                                              tweet sentiment  \
0   5421                              

In [70]:
#shuffle the datasets
en_dataset_binary = en_dataset_binary.sample(frac=1, random_state=42)
en_dataset=en_dataset.sample(frac=1, random_state=42)

In [49]:
#One by One

used_data = en_dataset_binary.iloc[0:40]
predictions_concat=[]

# Wrap range with tqdm
for i in tqdm(range(used_data.shape[0])):
    tweet = en_dataset['tweet'].iloc[i]
    # prompt
    prompt = [{"role": "system", "content": "Do you think these tweets are hatefull speech or just offensive speech. Try your best even if we do not have context. Display the reasult as an integer of 0 (Offensive) or 1 (Hatefull). Do not display anything else. Give an answer of this form even if you are not sure."},
              #{"role": "system", "content": "Do you think these tweets are hatefull speech, offensive or normal speech. Try your best to distinguish the 3 even if we do not have context. Display the reasult as an integer of 1 (Normal), 2(offensive) or 3 (Hateful). Do not display anything else, it is important that only the interger is displayed."},
              {"role": "user", "content": tweet},]
    # LLM initialization
    llm = Ollama(model="llama3")
    predictions = llm.invoke(prompt)
    predictions = int(predictions) 
    predictions_concat.append(predictions)

print(len(predictions_concat))
#add the predictions to the dataset
used_data['predictions'] = predictions_concat

#Metrics
acc=accuracy_score(used_data['sentiment_binary'], used_data['predictions'])
F1=f1_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
Precision=precision_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
Recall=recall_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
print(f"Accuracy: {acc:.3f}, F1: {F1:.3f}, Precision: {Precision:.3f}, Recall: {Recall:.3f}")


100%|██████████| 40/40 [01:15<00:00,  1.88s/it]
/var/folders/gm/941r48zd5qxcl4tb3v9q30440000gn/T/ipykernel_41400/570030429.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_data['predictions'] = predictions_concat


40


In [85]:
#other models
used_data = en_dataset_binary.iloc[0:40]
integers = []

# Wrap range with tqdm
for i in tqdm(range(used_data.shape[0])):
    tweet = en_dataset['tweet'].iloc[i]
    # prompt
    prompt = [{"role": "system", "content": "Classify tweets as hatefull speech or just offensive speech."},
              #{"role": "system", "content": "Do you think these tweets are hatefull speech, offensive or normal speech. Try your best to distinguish the 3 even if we do not have context. Display the reasult as an integer of 1 (Normal), 2(offensive) or 3 (Hateful). Do not display anything else, it is important that only the interger is displayed."},
              {"role": "user", "content": "Do you think this tweet is offensive (0) or hatefull (1) speech ? '{}' Try to make the distinction between offensive and hatefull speech even if you dont have any context. For your answer format, display 0 for offensive and 1 for hatefull speech at the begining of your answer. Then explains the reasoning behind this choice ".format(tweet)},]
    # LLM initialization
    llm = Ollama(model="llama3")
    predictions = llm.invoke(prompt)

    match = re.match(r'^[0-3]', predictions)

    if match:
        integers.append(int(match.group()))
            

print(len(integers))
#add the predictions to the dataset
used_data['predictions'] = integers

#Metrics
acc=accuracy_score(used_data['sentiment_binary'], used_data['predictions'])
F1=f1_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
Precision=precision_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
Recall=recall_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
print(f"Accuracy: {acc:.3f}, F1: {F1:.3f}, Precision: {Precision:.3f}, Recall: {Recall:.3f}")


  2%|▎         | 1/40 [01:16<49:24, 76.00s/it]


KeyboardInterrupt: 

In [80]:
#test
#print(used_data[[ 'sentiment','sentiment_binary', 'predictions']])
integers
predictions


'1\n\nI classified this tweet as hateful speech because it contains a derogatory term ("dyke") that is often used to harass or belittle LGBTQ+ individuals, particularly women. The use of a slur in a way that implies ridicule and disdain towards someone trying to carry groceries in their car is not just offensive, but also perpetuates harmful and discriminatory attitudes towards the community. The tone of the tweet suggests a clear intent to demean and degrade, which goes beyond mere offense.'

In [34]:
print(en_dataset_binary['sentiment_binary'].iloc[0:40])
print(en_dataset.shape)


419    1
565    1
284    0
469    1
349    0
588    1
369    0
234    0
212    0
587    1
535    1
596    1
368    0
539    1
556    1
437    1
595    1
257    0
453    1
515    1
441    1
275    0
364    0
506    1
570    1
473    1
490    1
245    0
248    0
555    1
591    1
356    0
209    0
329    0
536    1
548    1
561    1
589    1
316    0
304    0
Name: sentiment_binary, dtype: int64
(600, 8)
